# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 28/03/2024

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os
import concurrent.futures
import timeit

import matplotlib
%matplotlib widget

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_slice_recon, write_h5, read_h5
import ImageD11.columnfile
from ImageD11.sinograms import properties, roi_iradon
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.blobcorrector import eiger_spatial
from ImageD11.grain import grain
from ImageD11 import cImageD11
from xfab.parameters import read_par_file

from ImageD11.nbGui import nb_utils as utils

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)
   
sample = ds.sample
dataset = ds.dsname
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# load phases from parameter file

phases = ds.get_phases_from_disk()
ds.phases = phases
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json
major_phase_str = 'Fe'
minor_phase_str = 'Au'

major_phase_unitcell = ds.phases.unitcells[major_phase_str]
minor_phase_unitcell = ds.phases.unitcells[minor_phase_str]

print(major_phase_str, major_phase_unitcell.lattice_parameters, major_phase_unitcell.spacegroup)
print(minor_phase_str, minor_phase_unitcell.lattice_parameters, minor_phase_unitcell.spacegroup)

In [ ]:
tensor_map_major = TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + major_phase_str)
tensor_map_minor = TensorMap.from_h5(ds.grainsfile, h5group='TensorMap_' + minor_phase_str)

In [ ]:
tensor_map_major.plot('labels')
tensor_map_minor.plot('labels')

In [ ]:
tensor_map_combined = TensorMap.from_combine_phases([tensor_map_major, tensor_map_minor])

In [ ]:
tensor_map_combined.plot('phase_ids')
tensor_map_combined.plot('labels')
tensor_map_combined.plot('ipf_z')

In [ ]:
tensor_map_combined.to_h5(os.path.join(ds.analysispath, 'combined_map.h5'))
tensor_map_combined.to_paraview(os.path.join(ds.analysispath, 'combined_map.h5'))